In [148]:
import pandas as pd
import numpy as np
import math

class TreeNode: 
    def __init__(self, label=None, attributes=None, children=None):
        self.value = label  # value of the node
        self.attributes = attributes if attributes is not None else {}  # attributes in dict
        self.children = children or []  # dict of child nodes

    def __str__(self, level=0):
        prefix = "  " * level
        result = prefix + f"Attribute: {self.attributes}, Label: {self.value}\n"
        for child in self.children:
            result += prefix + f"Child:\n"
            result += child.__str__(level + 1)
        return result
    
    # def __str__(self):
    #     return str(self.value)

    def add_child(self, child_node):
        self.children.append(child_node)

def find_depth(node):
    # Base case: If the node has no children, its depth is 0
    if not node.children:
        return 0
    else:
        # Find the depth of each child node recursively
        child_depths = [find_depth(child) for child in node.children]
        # Return the maximum depth among child nodes + 1 (for the current node)
        return max(child_depths) + 1
    

# Example usage:
root = TreeNode("Root")
child1 = TreeNode("Child 1")
child2 = TreeNode("Child 2")
grandchild1 = TreeNode("Grandchild 1")
grandchild2 = TreeNode("Grandchild 2")

root.add_child(child1)
root.add_child(child2)
child1.add_child(grandchild1)
child2.add_child(grandchild2)

# Calculate the depth of the root node
root_depth = find_depth(root)
print("Depth of the root node:", root_depth)  # Should print 2

Depth of the root node: 2


In [149]:
# Find most common label
def find_most_common_label(labels):
    label_counts = count_elements(labels)
    print("label_counts: ", label_counts)
    most_common_label = max(label_counts, key=lambda k: label_counts[k])
    return most_common_label

def count_elements(lst):
    element_counts = {}
    for element in lst:
        if element in element_counts:
            element_counts[element] += 1
        else:
            element_counts[element] = 1
    return element_counts

In [150]:
#Test most common label
Label = ["-", "-", "+", "+", "+", "-", "+", "-", "+", "+", "+", "-", "+", "-"]

test_most_common_label = find_most_common_label(Label)
print(test_most_common_label)

label_counts:  {'-': 6, '+': 8}
+


In [151]:
df_S['Outlook'].unique()

array(['Sunny', 'Overcast', 'Rain'], dtype=object)

In [152]:
import math

def calculate_target_entropy(labels, class_list):
    #labels: the list of the label values in all data set, eg. df.iloc[:,-1].tolist()
    # class_list: the unique class list in target variable
    
    target_entropy = 0.0
    total_count = len(labels)
    
    for class_label in class_list:
        class_count = labels.count(class_label)
        print("class_label: ", class_label)
        print("class_count: ", class_count)
        class_probability = class_count / total_count
        if class_probability > 0:
            class_entropy = -class_probability * math.log2(class_probability)
            target_entropy += class_entropy
    
    return target_entropy

# TEST
Label = ["-", "-", "+", "+", "+", "-", "+", "-", "+", "+", "+", "+", "+", "-"]
class_list = ['-', '+']
test_target_entropy = calculate_target_entropy(Label, class_list)
print("Target Variable Entropy:", test_target_entropy)

class_label:  -
class_count:  5
class_label:  +
class_count:  9
Target Variable Entropy: 0.9402859586706311


In [153]:
def calculate_feature_entropy(feature_value_data, class_list):
    # feature_value_data: Subdataset with feature value data
    # class_list: the unique class list in target variable
    feature_total_count = feature_value_data.shape[0]
    label_list = feature_value_data.iloc[:, -1].tolist()
    print("feature_total_count: ", feature_total_count)
    entropy = 0
    
    for class_label in class_list:
        class_count = label_list.count(class_label)
        print("class_count: ", class_count)
        class_probability = class_count / feature_total_count
        if class_probability > 0:
            class_entropy = -class_probability * math.log2(class_probability)
            entropy += class_entropy

    return entropy

#test calulate_feature_entropy

S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"}
]
df_S = pd.DataFrame(S)
Label = ["-", "-", "-","+", "+"]
class_list = ["+", "-"]

test_feature_entropy = calculate_feature_entropy(df_S, class_list)
print(test_feature_entropy)

feature_total_count:  5
class_count:  2
class_count:  3
0.9709505944546686


In [154]:
def calculate_info_gain(feature_name, data, label, class_list):
    print("starting calculating {} information gain".format(feature_name))
    feature_value_list = data[feature_name].unique()
    print("feature_value_list: ",feature_value_list )
    total_row = data.shape[0]
    print("total_row: ", total_row)
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        print("starting feature_value={} in the list".format(feature_value))
        feature_value_data = data[data[feature_name] == feature_value] #filtering rows with that feature_value
        print("feature_value_data: ", feature_value_data)
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calculate_feature_entropy(feature_value_data, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calculate_target_entropy(label, class_list) - feature_info #calculating information gain by subtracting

def calculate_ME(feature_name, data, label, class_list):
    print("starting calculating {} information gain".format(feature_name))
    feature_value_list = data[feature_name].unique()
    print("feature_value_list: ",feature_value_list )
    total_row = data.shape[0]
    print("total_row: ", total_row)
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        print("starting feature_value={} in the list".format(feature_value))
        feature_value_data = data[data[feature_name] == feature_value] #filtering rows with that feature_value
        print("feature_value_data: ", feature_value_data)
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calculate_feature_entropy(feature_value_data, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calculate_target_entropy(label, class_list) - feature_info #calculating information gain by subtracting


# Sample Usage
S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

df_S = pd.DataFrame(S)
feature_name = 'Outlook'
Attributes = ["Outlook", "Temperature", "Humidity", "Wind"]
Label = ["-", "-", "+", "+", "+", "-", "+", "-", "+", "+", "+", "+", "+", "-"]
Target_var = 'Play'
class_list = ['-', '+']

test_gain = calculate_info_gain(feature_name, df_S, Label, class_list)
print("test gain: ", test_gain)

starting calculating Outlook information gain
feature_value_list:  ['Sunny' 'Overcast' 'Rain']
total_row:  14
starting feature_value=Sunny in the list
feature_value_data:     Outlook Temperature Humidity    Wind Play
0    Sunny         Hot     High    Weak    -
1    Sunny         Hot     High  Strong    -
7    Sunny        Mild     High    Weak    -
8    Sunny        Cool   Normal    Weak    +
10   Sunny        Mild   Normal  Strong    +
feature_total_count:  5
class_count:  3
class_count:  2
starting feature_value=Overcast in the list
feature_value_data:       Outlook Temperature Humidity    Wind Play
2   Overcast         Hot     High    Weak    +
6   Overcast        Cool   Normal  Strong    +
11  Overcast        Mild     High  Strong    +
12  Overcast         Hot   Normal    Weak    +
feature_total_count:  4
class_count:  0
class_count:  4
starting feature_value=Rain in the list
feature_value_data:     Outlook Temperature Humidity    Wind Play
3     Rain        Mild     High    Weak 

In [155]:

def find_most_informative_feature(data, label, class_list, purity_measurement='IG'):

    print("starting finding the most informative feature")
    print("data:",data)
    # Get the feature columns (all columns except the label column)
    feature_list = data.columns[:-1].tolist()
    print("feature_list: ", feature_list)
    max_info_gain = -1
    max_info_feature = None
    print("class_list: ", class_list)
    if purity_measurement == 'IG':
        print("calculate for information gain")
        print("starting calculating feaure gain")
        feature_info_gain = calculate_info_gain(feature, data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
        print("max info feature: ", max_info_feature)
    elif purity_measurement == 'majority':
        print("calculate as majority error")
        #TODO code for majority gain
    elif purity_measurement == 'gini':
        print("calculate as gini index")
        #TODO code for gini
    else:
        print("No valid purity measurement input.")
    for feature in feature_list:  #for each feature in the dataset
        print("starting calculating feaure gain")
        feature_info_gain = calculate_info_gain(feature, data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
        print("max info feature: ", max_info_feature)
            
    return max_info_feature

def choose_best_attribute(S, Attributes, Label, class_list, purity_measurement='IG'):
    print("starting choosing best attribute...")
    print("class_list: ", class_list)
    if purity_measurement == 'IG':
        print("starting use purity measurement as Information Gain")
        # Calculate information gain for all attributes and return the one with the maximum IG
        best_attribute = find_most_informative_feature(S, Label, class_list)
    # elif purity_measurement == 'majority':
    #     best_attribute = max(Attributes, key=lambda attr: calculate_me_gain(S, attr, Label))
    # elif purity_measurement == 'gini':
    #     best_attribute = max(Attributes, key=lambda attr: calculate_gini_gain(S, attr, Label))
    # else:
    #     print("Invalid purity measurement input. Only 'IG', 'majority', and 'gini' are supported. Defaulting to Information Gain.")
    #     best_attribute = max(Attributes, key=lambda attr: calculate_information_gain(S, attr, Label))

    return best_attribute

In [161]:

def ID3(S, Attributes, Label, purity_measurement=None):
    # S is a dataframe of the dataset
    # Attributes are the value of the attributes
    # Label are the list of the target variable for the datase
    #Default purity measurement for ID3
    
    if not purity_measurement:
        purity_measurement = 'IG'  # Default purity measurement
    print('purity_measurement:', purity_measurement)

    # Check if leaf mode with the same label
    unique_labels = set(Label)
    class_list = list(unique_labels)
    print("unique_labels: ", unique_labels)
    print("length of unique labels: ", len(unique_labels))
    print("class_list: ", class_list)
    if len(unique_labels) == 1:
        print('unique labels == 1')
        return TreeNode(label=unique_labels.pop())
    # Check if attribute is empty
    elif not Attributes:
        most_common_label = find_most_common_label(Label)
        print("attibute is empty, find most common label: ", most_common_label)
        return TreeNode(label=most_common_label)
    else:
        # Create a Root Node for tree
        root = TreeNode()
        print("starting create root node")
        # Choose the best attribute A to split S
        # Support the input purity measurement of IG, majorty and gini. Here we set default is IG.
        best_attribute = choose_best_attribute(S, Attributes, Label, class_list,purity_measurement)
        print("best attribute: ", best_attribute)
        root.attributes = best_attribute

        # Remove the chosen attribute from the list of attributes
        remaining_attributes = [attr for attr in Attributes if attr != best_attribute]
        print("remaining_attributes: ", remaining_attributes)

        # Split S into subsets based on the values of the best attribute
        attribute_values = S[best_attribute].unique()
        print("choosen attribute_values: ", attribute_values)

        # deal with the remaining attributes for subset Sv, according to A=V
        for value in attribute_values:
            print("value in attribute values:", value)
            print("best_attribute in attribute values:", best_attribute)
            # print(S)
            Sv = S[S[best_attribute] == value]
            remaining_label = Sv.iloc[:,-1].tolist()
            # print("value: {}, Sv {}".format(value, Sv))
            # If Sv is empty, add leaf node with the most common value of label in S
            if Sv.empty:
                most_common_label = find_most_common_label(Label)
                root.children[value] = TreeNode(label=most_common_label)
            else:
                return ID3(Sv, remaining_attributes, remaining_label, purity_measurement)
    return root

In [162]:
# test
best_attribute = 'Outlook'
Attributes = ["Outlook", "Temperature", "Humidity", "Wind"]
test_remaining_attributes = [attr for attr in Attributes if attr != best_attribute]
print(test_remaining_attributes)

['Temperature', 'Humidity', 'Wind']


In [163]:
# Sample Usage
S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

# S = [
#     {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
#     {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
#     {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
#     {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
#     {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"}
# ]
df_S = pd.DataFrame(S)

Attributes = ["Outlook", "Temperature", "Humidity", "Wind"]
Label = ["-", "-", "+", "+", "+", "-", "+", "-", "+", "+", "+", "+", "+", "-"]
Target_var = 'Play'


root = ID3(df_S, Attributes, Label)  # You can change purity_measurement as needed

purity_measurement: IG
unique_labels:  {'-', '+'}
length of unique labels:  2
class_list:  ['-', '+']
starting create root node
starting choosing best attribute...
class_list:  ['-', '+']
starting use purity measurement as Information Gain
starting finding the most informative feature
data:      Outlook Temperature Humidity    Wind Play
0      Sunny         Hot     High    Weak    -
1      Sunny         Hot     High  Strong    -
2   Overcast         Hot     High    Weak    +
3       Rain        Mild     High    Weak    +
4       Rain        Cool   Normal    Weak    +
5       Rain        Cool   Normal  Strong    -
6   Overcast        Cool   Normal  Strong    +
7      Sunny        Mild     High    Weak    -
8      Sunny        Cool   Normal    Weak    +
9       Rain        Mild   Normal    Weak    +
10     Sunny        Mild   Normal  Strong    +
11  Overcast        Mild     High  Strong    +
12  Overcast         Hot   Normal    Weak    +
13      Rain        Mild     High    Weak    -
fea

In [164]:
root.attributes

{}

In [165]:
print(root)

Attribute: {}, Label: -



In [135]:
# test
value = 'Sunny'
best_attribute = 'Outlook'
S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "-"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Play": "+"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
]

# S = [
#     {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "-"},
#     {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "-"},
#     {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "-"},
#     {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "+"},
#     {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Play": "+"}
# ]
# df_S = pd.DataFrame(S)
Sv = df_S[df_S[best_attribute] == value]

print(Sv)

   Outlook Temperature Humidity    Wind Play
0    Sunny         Hot     High    Weak    -
1    Sunny         Hot     High  Strong    -
7    Sunny        Mild     High    Weak    -
8    Sunny        Cool   Normal    Weak    +
10   Sunny        Mild   Normal  Strong    +


In [ ]:
S